<a href="https://colab.research.google.com/github/M-110/testing-with-pytest/blob/main/03_Pytest_Fixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Highlights:
* storing fixtures in conftest
* Fixtures as setup/teardown
* Fixture scopes (class, module, etc)
* Auto-use fixtures

# Simple data fixture

In [ ]:
%%writefile test_fixtures.py
import pytest

@pytest.fixture()
def some_data():
  return 42


def test_some_data(some_data):
  assert some_data == 42

Writing test_fixtures.py


In [ ]:
!pytest test_fixtures.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_fixtures.py::test_some_data PASSED                                  [100%]

=========================== 1 passed in 0.02 seconds ===========================


In [ ]:
%%writefile test_fixtures2.py
import pytest

@pytest.fixture()
def arg1():
  return (2, 2)


@pytest.fixture()
def arg2():
  return 5



def test_some_data(arg1, arg2):
  a, b = arg1
  assert a + b == arg2

Writing test_fixtures2.py


In [ ]:
!pytest test_fixtures2.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_fixtures2.py::test_some_data FAILED                                 [100%]

=================================== FAILURES ===================================
________________________________ test_some_data ________________________________

arg1 = (2, 2), arg2 = 5

    def test_some_data(arg1, arg2):
      a, b = arg1
>     assert a + b == arg2
E     assert (2 + 2) == 5

test_fixtures2.py:16: AssertionError
=========================== 1 failed in 0.03 seconds ===========================


# Storing fixtures in confest

In [ ]:
%%writefile conftest.py
import pytest

@pytest.fixture()
def arg1():
  return (2, 3)


@pytest.fixture()
def arg2():
  return 5

Writing conftest.py


In [ ]:
%%writefile test_fixtures3.py


def test_some_conftest(arg1, arg2):
  a, b = arg1
  assert a + b == arg2

Writing test_fixtures3.py


In [ ]:
%%writefile test_fixtures4.py


def test_some_conftest(arg1, arg2):
  a, b = arg1
  assert a - b == arg2

Writing test_fixtures4.py


In [ ]:
!pytest -v -k conftest

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 6 items / 4 deselected                                               

test_fixtures3.py::test_some_conftest PASSED                             [ 50%]
test_fixtures4.py::test_some_conftest FAILED                             [100%]

=================================== FAILURES ===================================
______________________________ test_some_conftest ______________________________

arg1 = (2, 3), arg2 = 5

    def test_some_conftest(arg1, arg2):
      a, b = arg1
>     assert a - b == arg2
E     assert (2 - 3) == 5

test_fixtures4.py:5: AssertionError
=============== 1 failed, 1 passed, 4 deselected in 0.06 seconds ===============


# Fixtures to setup and teardown a database

In [ ]:
%%writefile conftest.py
import pytest

@pytest.fixture()
def my_db():
  # Creating database
  db = 'my_db'
  yield db
  del db
  # Destroying database

Overwriting conftest.py


In [ ]:
%%writefile test_fixtures5.py


def test_some_conftest(my_db):
  assert 5 == 5

Writing test_fixtures5.py


In [ ]:
!pytest --setupshow -v test_fixtures5.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_fixtures5.py::test_some_conftest 
      SETUP    F my_db
        test_fixtures5.py::test_some_conftest (fixtures used: my_db)PASSED
      TEARDOWN F my_db

=========================== 1 passed in 0.01 seconds ===========================


# Fixtures as a tuple

In [ ]:
%%writefile test_fixtures6.py
import pytest


@pytest.fixture()
def a_tuple():
  return (1, 'dog', None, {'cat':  5})


def test_a_tuple(a_tuple):
  assert a_tuple[3]['cat'] == 4

Writing test_fixtures6.py


In [ ]:
!pytest --setupshow -v test_fixtures6.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_fixtures6.py::test_a_tuple 
      SETUP    F a_tuple
        test_fixtures6.py::test_a_tuple (fixtures used: a_tuple)FAILED
      TEARDOWN F a_tuple

=================================== FAILURES ===================================
_________________________________ test_a_tuple _________________________________

a_tuple = (1, 'dog', None, {'cat': 5})

    def test_a_tuple(a_tuple):
>     assert a_tuple[3]['cat'] == 4
E     assert 5 == 4

test_fixtures6.py:10: AssertionError
=========================== 1 failed in 0.03 seconds ===========================


# Fixtures failing

In [ ]:
%%writefile test_fixtures7.py
import pytest


@pytest.fixture()
def a_tuple():
  assert True == False
  return (1, 'dog', None, {'cat':  5})


def test_a_tuple(a_tuple):
  assert a_tuple[3]['cat'] == 4

Writing test_fixtures7.py


In [ ]:
!pytest --setupshow -v test_fixtures7.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_fixtures7.py::test_a_tuple 
      SETUP    F a_tupleERROR
      TEARDOWN F a_tuple

==================================== ERRORS ====================================
________________________ ERROR at setup of test_a_tuple ________________________

    @pytest.fixture()
    def a_tuple():
>     assert True == False
E     assert True == False

test_fixtures7.py:6: AssertionError
=========================== 1 error in 0.03 seconds ============================


# Combining fixtures

In [ ]:
%%writefile conftest.py
import pytest

@pytest.fixture()
def my_db():
  # Creating database
  db = 'my_db: '
  yield db
  del db
  # Destroying database

@pytest.fixture()
def multiple_names():
  return ['Amy', 'Aaron', 'Adam']


@pytest.fixture()
def db_with_multiple_names(my_db, multiple_names):
  for name in multiple_names:
    my_db += name
  return my_db

Overwriting conftest.py


In [ ]:
%%writefile test_fixtures8.py

def test_db_stuff(db_with_multiple_names):
  assert 'Zach' in db_with_multiple_names

def test_other_db_stuff(db_with_multiple_names):
  assert 'Amy' in db_with_multiple_names


Writing test_fixtures8.py


In [ ]:
!pytest test_fixtures8.py --setup-show -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items                                                              

test_fixtures8.py::test_db_stuff 
      SETUP    F my_db
      SETUP    F multiple_names
      SETUP    F db_with_multiple_names (fixtures used: multiple_names, my_db)
        test_fixtures8.py::test_db_stuff (fixtures used: db_with_multiple_names, multiple_names, my_db)FAILED
      TEARDOWN F db_with_multiple_names
      TEARDOWN F multiple_names
      TEARDOWN F my_db
test_fixtures8.py::test_other_db_stuff 
      SETUP    F my_db
      SETUP    F multiple_names
      SETUP    F db_with_multiple_names (fixtures used: multiple_names, my_db)
        test_fixtures8.py::test_other_db_stuff (fixtures used: db_with_multiple_names, multiple_names, my_db)PASSED
      TEARDOWN F db_with_multipl

# Fixture scopes

In [ ]:
%%writefile conftest.py
import pytest

@pytest.fixture(scope='function')
def my_func_stuff():
  cache = {}
  yield cache
  del cache

@pytest.fixture(scope='class')
def my_class_stuff():
  class_info = []
  yield class_info
  del class_info

@pytest.fixture(scope='module')
def my_module_stuff():
  module_things = set()
  yield module_things
  del module_things

@pytest.fixture(scope='session')
def my_session_stuff():
  session_stuff = {}
  yield session_stuff
  del session_stuff


Overwriting conftest.py


In [ ]:
%%writefile test_fixture_scope1.py
import pytest

def test_a(my_func_stuff, my_module_stuff, my_session_stuff):
  assert 1 == 1

@pytest.mark.usefixtures('my_class_stuff', 'my_module_stuff', 'my_session_stuff')
class TestB:
  def test_c(self):
    assert 1 == 1

  def test_d(self):
    assert 1 == 1

Writing test_fixture_scope1.py


In [ ]:
%%writefile test_fixture_scope2.py
def test_e(my_module_stuff, my_session_stuiff, my_func_stuff):
  assert 1 == 1

Writing test_fixture_scope2.py


In [ ]:
!pytest -k scope --setup-show 

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 15 items / 9 deselected                                              

test_fixture_scope1.py 
SETUP    S my_session_stuff
  SETUP    M my_module_stuff
      SETUP    F my_func_stuff
        test_fixture_scope1.py::test_a (fixtures used: my_func_stuff, my_module_stuff, my_session_stuff).
      TEARDOWN F my_func_stuff
    SETUP    C my_class_stuff
        test_fixture_scope1.py::TestB::()::test_c (fixtures used: my_class_stuff, my_module_stuff, my_session_stuff).
        test_fixture_scope1.py::TestB::()::test_d (fixtures used: my_class_stuff, my_module_stuff, my_session_stuff).
    TEARDOWN C my_class_stuff
  TEARDOWN M my_module_stuff                                             [  0%]
test_fixture_scope2.py 
  SETUP    M my_module_stuffE
  TEARDOWN M my_module_stuff                                

# You can use autouse for fixtures that always get used

In [ ]:
%%writefile test_auto.py
import pytest
import time

@pytest.fixture(autouse=True,scope='session')
def footer_session_scope():
  yield
  now = time.time()
  print('------')
  print('Finshed: ', now)
  print('-------')

@pytest.fixture(autouse=True)
def footer_function_scope():
  start = time.time()
  yield
  stop = time.time()
  delta = stop - start
  print('\ntest duration:', delta, 'seconds')


# The fixtures apply to both even though you didn't specify them.
def test_1():
  time.sleep(1)

def test_2():
  time.sleep(1.23)
  

Overwriting test_auto.py


In [ ]:
!pytest test_auto.py -v -s

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 2 items                                                              

test_auto.py::test_1 PASSED('\ntest duration:', 1.001974105834961, 'seconds')

test_auto.py::test_2 PASSED('\ntest duration:', 1.2321350574493408, 'seconds')
------
('Finshed: ', 1626801695.576027)
-------


=========================== 2 passed in 2.24 seconds ===========================


Autouse is good, but generally should be avoided.

# Renaming Fixtures

In [ ]:
%%writefile test_rename.py
import pytest

# You can use 'name' instead of the actualy function name
@pytest.fixture(name='answer')
def answer_but_with_a_really_long_excessive_naming_choice():
  """Return the answer."""
  return 42


def test_everything(answer):
  assert answer == 42


Overwriting test_rename.py


In [ ]:
!pytest test_rename.py -v --setup-show

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 1 item                                                               

test_rename.py::test_everything 
      SETUP    F answer
        test_rename.py::test_everything (fixtures used: answer)PASSED
      TEARDOWN F answer

=========================== 1 passed in 0.02 seconds ===========================


In [ ]:
!pytest --fixtures test_rename.py

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               
cache
    Return a cache object that can persist state between testing sessions.
    
    cache.get(key, default)
    cache.set(key, value)
    
    Keys must be a ``/`` separated value, where the first part is usually the
    name of your plugin or application to avoid clashes with other cache users.
    
    Values can be any object handled by the json stdlib module.
capsys
    Enable capturing of writes to ``sys.stdout`` and ``sys.stderr`` and make
    captured output available via ``capsys.readouterr()`` method calls
    which return a ``(out, err)`` namedtuple.  ``out`` and ``err`` will be ``text``
    objects.
capsysbinary
    Enable capturing of writes to ``sys.stdout`` and ``sys.stderr`` and make
    captured output availa

# Parametrizing Fixtures

In [ ]:
%%writefile test_params.py
import pytest

class Cat:
  def __init__(self, name, age, power):
    self.name = name
    self.age = age
    self.power = power
  
  def __repr__(self):
    return self.name

cats = [Cat('felix', 50, 9),
        Cat('pete', 12, 10),
        Cat('sylvester', 35, 4)]


@pytest.fixture(params=cats)
def some_cat(request):
  return request.param

def test_add_some_cats(some_cat):
 
  assert some_cat.name == 'pete'

Overwriting test_params.py


In [ ]:
!pytest test_params.py -v -s --setup-show

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 3 items                                                              

test_params.py::test_add_some_cats[some_cat0] 
      SETUP    F some_cat[felix]
        test_params.py::test_add_some_cats[some_cat0] (fixtures used: some_cat)FAILED
      TEARDOWN F some_cat[felix]
test_params.py::test_add_some_cats[some_cat1] 
      SETUP    F some_cat[pete]
        test_params.py::test_add_some_cats[some_cat1] (fixtures used: some_cat)PASSED
      TEARDOWN F some_cat[pete]
test_params.py::test_add_some_cats[some_cat2] 
      SETUP    F some_cat[sylvester]
        test_params.py::test_add_some_cats[some_cat2] (fixtures used: some_cat)FAILED
      TEARDOWN F some_cat[sylvester]

=================================== FAILURES ===================================
__________

# Exercises

* Create a test file called test_fixtures.py.
 
* Write a few data fixtures—functions with the @pytest.fixture() decorator—that r eturn some data. Perhaps a list, or a dictionary, or a tuple.
 
* For each fixture, write at least one test function that uses it.
 
* Write two tests that use the same fixture.
 
* Run pytest --setup-show test_fixtures.py. Are all the fixtures run before e very test?
 
* Add scope=’module’ to the fixture from Exercise 4.
 
* Re-run pytest --setup-show test_fixtures.py. What changed?
 
* For the fixture from Exercise 6, change return <data> to yield <data>.
 
* Add print statements before and after the yield.
 
* Run pytest -s -v test_fixtures.py. Does the output make sense?

In [ ]:
%%writefile test_fixtures.py
import pytest

@pytest.fixture()
def some_lists():
  return ([1, 2, 3],
          [4, 5, 6],
          [7, 8, 9])

@pytest.fixture()
def some_dicts():
  return (dict(name='paul',
              location='home'),
          dict(name='sasha',
               location='bank'))

@pytest.fixture(scope='module')
def some_names():
  return ('mildrid', 'silus', 'kwame')

def test_dicts(some_dicts, some_names):
  assert some_dicts[0]['name'] in ('paul', 'sasha')

def test_lists(some_lists):
  assert some_lists[0][1] == 2


def test_names(some_names):
  assert 'i' in some_names[0]

Overwriting test_fixtures.py


In [ ]:
!pytest test_fixtures.py -v --setup-show

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 3 items                                                              

test_fixtures.py::test_dicts 
  SETUP    M some_names
      SETUP    F some_dicts
        test_fixtures.py::test_dicts (fixtures used: some_dicts, some_names)PASSED
      TEARDOWN F some_dicts
test_fixtures.py::test_lists 
      SETUP    F some_lists
        test_fixtures.py::test_lists (fixtures used: some_lists)PASSED
      TEARDOWN F some_lists
test_fixtures.py::test_names 
        test_fixtures.py::test_names (fixtures used: some_names)PASSED
  TEARDOWN M some_names

=========================== 3 passed in 0.03 seconds ===========================


In [ ]:
%%writefile test_fixtures2.py
import pytest

@pytest.fixture()
def some_lists():
  return ([1, 2, 3],
          [4, 5, 6],
          [7, 8, 9])

@pytest.fixture()
def some_dicts():
  return (dict(name='paul',
              location='home'),
          dict(name='sasha',
               location='bank'))

@pytest.fixture(scope='module')
def some_names():
  print('CREATING NAMES')
  yield ('mildrid', 'silus', 'kwame')
  print('DESTROYING NAMES')

def test_dicts(some_dicts, some_names):
  assert some_dicts[0]['name'] in ('paul', 'sasha')

def test_lists(some_lists):
  assert some_lists[0][1] == 2


def test_names(some_names):
  assert 'i' in some_names[0]

Overwriting test_fixtures2.py


In [ ]:
!pytest test_fixtures2.py -v --setup-show -s

============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1 -- /usr/bin/python2
cachedir: .pytest_cache
rootdir: /content, inifile:
collected 3 items                                                              

test_fixtures2.py::test_dicts CREATING NAMES

  SETUP    M some_names
      SETUP    F some_dicts
        test_fixtures2.py::test_dicts (fixtures used: some_dicts, some_names)PASSED
      TEARDOWN F some_dicts
test_fixtures2.py::test_lists 
      SETUP    F some_lists
        test_fixtures2.py::test_lists (fixtures used: some_lists)PASSED
      TEARDOWN F some_lists
test_fixtures2.py::test_names 
        test_fixtures2.py::test_names (fixtures used: some_names)PASSEDDESTROYING NAMES

  TEARDOWN M some_names

=========================== 3 passed in 0.03 seconds ===========================
